# ARIMA(p d q)

https://medium.com/@kis.andras.nandor/understanding-autocorrelation-and-partial-autocorrelation-functions-acf-and-pacf-2998e7e1bcb5

https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting#7.-How-to-find-the-order-of-the-MA-term-(q)-

https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

https://www.geeksforgeeks.org/python-arima-model-for-time-series-forecasting/

## Setup

In [0]:
!pip install pmdarima

In [0]:
%restart_python 

In [0]:
spark.sql('use catalog dbacademy')
spark.sql('use schema labuser9128531_1738705451')

In [0]:
path='/Volumes/dbacademy/labuser9128531_1738705451/air/AirPassengers.csv'
# Read the CSV file into a DataFrame
df = spark.read.csv(path, header=True, inferSchema=True)

In [0]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [0]:
df=df.toPandas()

In [0]:
df.info()

In [0]:
df['Date']=pd.to_datetime(df['Date'])

## ADF test for data stationary
If p>0.05, meaning the data is not stationary 

In [0]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(df.Passengers.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

## Visualize the data

For the above data, we can see that the time series reaches stationarity with two orders of differencing.

**Autoregression** 
The Autocorrelation Function (ACF)
The ACF plots the correlation of the time series with itself at different lags. This helps in identifying patterns such as seasonality, trends, and the persistence of values over time.

Lag 1: Correlation between observations at time t and t−1
Lag 2: Correlation between observations at time t and t−2

When you look at an ACF plot, you’ll see bars at each lag. The height of the bar represents the correlation coefficient at that lag.

Significant Lag: If a bar extends beyond the significance bounds, it indicates significant autocorrelation at that lag.the ACF plot is also useful for identifying non-stationary time series. For a stationary time series, the ACF will drop to zero relatively quickly, while the ACF of non-stationary data decreases slowly.

`Gradual Decline`: A gradual decline in bar heights suggests a long-term dependency in the data.

`Seasonal Patterns`: Regular spikes at certain lags suggest seasonality in the data.

**Stationary** properties do not depend on the time at which the series is observed.time series with trends, or with seasonality, are not stationary — the trend and seasonality will affect the value of the time series at different times. On the other hand, a white noise series is stationary — it does not matter when you observe it, it should look much the same at any point in time.

Differencing: a method to get to a stationary: constant mean and variance
Differencing can help stabilise the mean of a time series by removing changes in the level of a time series, and therefore eliminating (or reducing) trend and seasonality.

In [0]:
df.Passengers.head(10)

In [0]:
len(df)

In [0]:
df.Passengers.diff().head(10)

In [0]:
df.Passengers.diff().diff().diff().diff()

In [0]:
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

In [0]:
df.Passengers.diff().diff().diff().diff().dropna().plot()

In [0]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams.update({'figure.figsize':(9,20), 'figure.dpi':120})

In [0]:
# Original Series
fig, axes = plt.subplots(5, 2, sharex=False)
axes[0, 0].plot(df.Passengers); axes[0, 0].set_title('Original Series')
plot_acf(df.Passengers,lags=len(df.Passengers)//2, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df.Passengers.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(df.Passengers.diff().dropna(), lags=len(df.Passengers)//2,ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(df.Passengers.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(df.Passengers.diff().diff().dropna(),lags=len(df.Passengers)//2, ax=axes[2, 1])

# 3rd Differencing
axes[3, 0].plot(df.Passengers.diff().diff().diff()); axes[3, 0].set_title('3rd Order Differencing')
plot_acf(df.Passengers.diff().diff().diff().dropna(),lags=len(df.Passengers)//2, ax=axes[3, 1])
plt.show()

# 4th Differencing
axes[4, 0].plot(df.Passengers.diff().diff().diff().diff()); axes[4, 0].set_title('4th Order Differencing')
plot_acf(df.Passengers.diff().diff().diff().diff().dropna(), ax=axes[4, 1])
plt.show()

## find the order of the AR term (p) using PACF
find out the required number of AR terms by inspecting the Partial Autocorrelation (PACF) plot.

Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. This way, we will know if that lag is needed in the AR term or not.

The PACF plot shows the partial correlation of the time series with itself at different lags.
The PACF plot helps determine the order of an autoregressive model (AR model).

Significant Lag: A significant spike at a particular lag suggests the inclusion of that lag in the AR model.
Cut-off Point: The lag at which the PACF plot cuts off helps determine the maximum lag to include in the AR model.

In [0]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,6), 'figure.dpi':120})

fig, axes = plt.subplots(2, 2, sharex=False)
axes[0,0].plot(df.Passengers); axes[0,0].set_title('Original Series')
axes[0,1].set(ylim=(0,5))
plot_pacf(df.Passengers, ax=axes[0,1])

axes[1,0].plot(df.Passengers.diff()); axes[1,0].set_title('1st Differencing')
axes[1,1].set(ylim=(0,5))
plot_pacf(df.Passengers.diff().dropna(), ax=axes[1,1])

plt.show()

We can see that the PACF lag 1 and 2 is quite significant after the 1st differencing since it is well above the significance line. So, we will fix the value of p as 2.

## Find out the MA (q) suing ACF
Just like how we looked at the PACF plot for the number of AR terms, we will look at the ACF plot for the number of MA terms. An MA term is technically, the error of the lagged forecast.
The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series.

The ACF plots the correlation of the time series with itself at different lags. This helps in identifying patterns such as `seasonality`, `trends`, and the `persistence` of values over time.

Lag 1: Correlation between observations at time t and t−1

Lag 2: Correlation between observations at time t and t−2

The height of the bar represents the correlation coefficient at that lag.

Significant Lag: If a bar extends beyond the significance bounds, it indicates significant autocorrelation at that lag.
Gradual Decline: A gradual decline in bar heights suggests a long-term dependency in the data.
Seasonal Patterns: Regular spikes at certain lags suggest seasonality in the data.

In [0]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(df.Passengers.diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(df.Passengers.diff().dropna(),lags=130, ax=axes[1])

plt.show()

We can see that couple of lags are well above the significance line. So, we will fix q as 2. If there is any doubt, we will go with the simpler model that sufficiently explains the Y.

If the series is slightly under differenced, adding one or more additional AR terms usually makes it up. Likewise, if it is slightly over-differenced, we will try adding an additional MA term.

## Build the ARIMA Model 
CIA should be low and p for the last ar or ma shoud be lower than 0.05

### Compare different models

In [0]:
from statsmodels.tsa.arima.model import ARIMA

# 1,1,2 ARIMA Model
model = ARIMA(df.Passengers, order=(1,1,2))
model_fit = model.fit()
print(model_fit.summary())

In [0]:
model = ARIMA(df.Passengers, order=(2,1,2))
model_fit = model.fit()
print(model_fit.summary())

In the second model, AIC is reduced and p also reduced slightly. So the 2nd looks better

In [0]:
model = ARIMA(df.Passengers, order=(2,0,1))
model_fit = model.fit()
print(model_fit.summary())

### PLot residuals
look for constant mean and variance, meaning there are no pattern

In [0]:
# Plot residual errors
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

The residual errors seem fine with near zero mean and uniform variance. Let’s plot the actuals against the fitted values using plot_predict().

### PLot predicted values

When we set dynamic=False the in-sample lagged values are used for prediction. That is, the model gets trained up until the previous value to make the next prediction. This can make the fitted forecast and actuals look artificially good.

In [0]:
from statsmodels.graphics.tsaplots import plot_predict

# Actual vs Fitted
plot_predict(model_fit, start=10, end=len(df)-1,dynamic=False)
df.Passengers.plot()
plt.show()

## Find the optimal ARIMA model using Out-of-Time Cross validation 
In Out-of-Time cross-validation, we move backwards in time and forecast into the future to as many steps we took back. Then we compare the forecast against the actuals.

To do so, we will create the training and testing dataset by splitting the time series into 2 contiguous parts in a reasonable proportion based on time frequency of series.

In [0]:
len(df)

### Create train and test samples

In [0]:
# Create Training and Test
train = df.Passengers[:85]
test = df.Passengers[85:]

In [0]:
from statsmodels.tsa.stattools import acf

In [0]:
len(test)

### Create model on training data

In [0]:
# Build Model
# model = ARIMA(train, order=(3,1,2))  
model = ARIMA(train, order=(2, 2, 1))  # middle is d 
fitted = model.fit()  

In [0]:
fitted.summary()

### PLot prediced and observed values

In [0]:
# Actual vs Fitted
plot_predict(fitted, start=10, end=len(df)-1,dynamic=False)
df.Passengers.plot()
plt.show()

### Accuracy Metrics for Time Series Forecast 

In [0]:
fc= fitted.forecast(59, alpha=0.05)  # 95% conf

In [0]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test)

Around 15.20% MAPE implies the model is about 85% accurate in predicting the next 15 observations. Now we know how to build an ARIMA model manually.

## Auto ARIMA Forecasting

### Comapre models

In [0]:
from statsmodels.tsa.arima_model import ARIMA

import pmdarima as pm

In [0]:
model = pm.auto_arima(df.Passengers, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [0]:
print(model.summary())

### Interpret the residual plots in ARIMA model

Interpretation of plots in plot diagnostics
`Standardized residual`: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

`Histogram`: The density plot suggest normal distribution with mean slighlty shifted towards right.

`Theoretical Quantiles`: Mostly the dots fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

`Correlogram`: The Correlogram, (or ACF plot) shows the residual errors are not autocorrelated. The ACF plot would imply that there is some pattern in the residual errors which are not explained in the model. So we will need to look for more X’s (predictors) to the model.
Overall, the model seems to be a good fit. So, let's use it to forecast.

In [0]:
import matplotlib.pyplot as plt
model.plot_diagnostics(figsize=(10,8))
plt.show()

### Model predict

In [0]:
# Forecast
n_periods = 24
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df.Passengers), len(df.Passengers)+n_periods)

In [0]:
confint

In [0]:
index_of_fc

In [0]:
# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(df.Passengers)
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast of Usage")
plt.show()

## SARIMA model
ARIMA does not support seasonality

### Test differencing to reduce easonality

Seasonal differencing is similar to regular differencing, but, instead of subtracting consecutive terms, we subtract the value from previous season.
So, the model will be represented as SARIMA(p,d,q)x(P,D,Q), where, P, D and Q are SAR, order of seasonal differencing and SMA terms respectively and 'x' is the frequency of the time series. If the model has well defined seasonal patterns, then enforce D=1 for a given frequency ‘x’.

We should set the model parameters such that D never exceeds one. And the total differencing ‘d + D’ never exceeds 2. We should try to keep only either SAR or SMA terms if the model has seasonal components.

In [0]:
df.head(5)

In [0]:
# Plot
fig, axes = plt.subplots(2, 1, figsize=(10,5), dpi=100, sharex=True)

# Usual Differencing
axes[0].plot(df.Date,df.Passengers, label='Original Series')
axes[0].plot(df.Date,df.Passengers.diff(1), label='Usual Differencing')
axes[0].set_title('Usual Differencing')
axes[0].legend(loc='upper left', fontsize=10)

# Seasonal Differencing
axes[1].plot(df.Date,df.Passengers, label='Original Series')
axes[1].plot(df.Date,df.Passengers.diff(12), label='Seasonal Differencing', color='green') #rectified
axes[1].set_title('Seasonal Differencing')
plt.legend(loc='upper left', fontsize=10)
plt.suptitle('AirPassengers - Time Series Dataset', fontsize=16)
plt.show()

We can see that, the seasonal spikes are intact after applying usual differencing (lag 1). Whereas, it is rectified after seasonal differencing.

### Create SARIMA model using auto_ARIMA
Based on the previous plot, we set seasonal=True, set the frequency m=12 for month wise series and enforce D=1.

In [0]:
import pmdarima as pm

In [0]:
# Seasonal - fit stepwise auto-ARIMA
smodel = pm.auto_arima(df.Passengers, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

smodel.summary()

### PLot values 

In [0]:
df.Date.max()

In [0]:
pd.date_range(start='1/1/1961', end='12/1/1962', freq='MS')

In [0]:
# Forecast
n_periods=24
fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = pd.date_range(start='1/1/1961', end='12/1/1962', freq='MS')

In [0]:
index_of_fc, len(index_of_fc)

In [0]:
fitted, len(fitted)

In [0]:
# make series for plotting purpose
fitted_series = pd.Series(list(fitted), index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

In [0]:
lower_series

In [0]:
fitted_series 

In [0]:
# Plot
plt.plot(df.Date, df.Passengers, color='blue')
plt.plot(fitted_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("SARIMA - Final Forecast of Drug Sales - Time Series Dataset")
plt.show()

### Using the train and test 

#### Create model using the parameters identified by auto_ARIMA

In [0]:
train=pd.DataFrame(train)

In [0]:
test=pd.DataFrame(test)

In [0]:
train.columns, train.index

In [0]:
train.head(5), test.head(5)

In [0]:
# Fit a SARIMAX(0, 1, 1)x(2, 1, 0, 12) on the training set 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
model_1 = SARIMAX(train['Passengers'],  
                order = (0, 1, 1),  
                seasonal_order =(2, 1, 0, 12)) 
  
result_1 = model_1.fit() 
result_1.summary()

#### Predict values in test data

In [0]:
start = len(train) 
end = len(train) + len(test) - 1
  
# Predictions for one-year against the test set 
predictions_1 = result_1.predict(start, end, 
                             typ = 'levels').rename("Predictions") 
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})  
# plot predictions and actual values 
predictions_1.plot(legend = True) 
test['Passengers'].plot(legend = True) 

#### Evaluate model

In [0]:
# Load specific evaluation tools 
from sklearn.metrics import mean_squared_error 
from statsmodels.tools.eval_measures import rmse 
  
# Calculate root mean squared error 
rmse(test["Passengers"], predictions_1) 

In [0]:
# Calculate mean squared error 
mean_squared_error(test["Passengers"], predictions_1) 

### Train whole data, predict future 3 years

In [0]:
len(df)/12

In [0]:
start,end

In [0]:
df.index[-1]

In [0]:
# Train the model on the full dataset 
model = model = SARIMAX(df['Passengers'],  
                        order = (0, 1, 1),  
                        seasonal_order =(2, 1, 0, 12)) 
result = model.fit() 
  
# Forecast for the next 3 years 
forecast = result.predict(start = len(df),  #144
                          end = (len(df)-1) + 3 * 12,  #179
                          typ = 'levels').rename('Forecast') 
  
# Plot the forecast values 
df['Passengers'].plot(figsize = (12, 5), legend = True) 
forecast.plot(legend = True)

## SARIMAX model with exogeneous variables

https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting#14.-SARIMA-model-in-python-

we will force an external predictor, also called, exogenous variable into the model. This model is called the SARIMAX model. The only requirement to use an exogenous variable is we should know the value of the variable during the forecast period as well.

I want to see how the model looks if we force the recent seasonality pattern into the training and forecast. The seasonal index is a good exogenous variable because it repeats every frequency cycle, 12 months in this case.

So, we will always know what values the seasonal index will hold for the future forecasts.

Let’s compute the seasonal index so that it can be forced as a (exogenous) predictor to the SARIMAX model.

### Compute Seasonal Index

In [0]:
# Compute Seasonal Index
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

In [0]:
df.head(5)

We have effectively forced the latest seasonal effect of the latest **3 years** into the model instead of the entire history.

In [0]:
# multiplicative seasonal component
df.set_index('Date', inplace=True)
result_mul = seasonal_decompose(df['Passengers'][-36:],   # 3 years
                                model='multiplicative', 
                                extrapolate_trend='freq')

seasonal_index = result_mul.seasonal[-12:].to_frame()
seasonal_index['month'] = pd.to_datetime(seasonal_index.index).month
seasonal_index

In [0]:
df.head(5)

In [0]:
df.index.month

In [0]:
# merge with the base data
df['month'] = df.index.month
#merge seasonality into the data
df1 = pd.merge(df, seasonal_index, how='left', on='month')
print(df1.head(5))
df1.columns = ['Passengers', 'month', 'seasonal_index']
print(df1.head(5))

In [0]:
df1.index = df.index  # reassign the index
print(df1.head(5))

### Create SARIMARX model

In [0]:
df1.info()

In [0]:
import pmdarima as pm

# SARIMAX Model
sxmodel = pm.auto_arima(df1[['Passengers']],
                           exogenous=df1[['seasonal_index']], # only one different from SARIMA
                           start_p=1, start_q=1,
                           test='adf',
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=None, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
sxmodel.summary()

We should look for **x1** in the first col along with ma r1, arsl12...
Here there is no this one, meaning the coefficient is close to 0, very small. It means it does not contribute to the model. 

### Forcast for future 24 months

In [0]:
seasonal_index.head(5)

In [0]:
#repeat an array for 2 time
np.tile(seasonal_index.seasonal, 2)

In [0]:
#repeat an array for 2 time and reshape it 
np.tile(seasonal_index.seasonal, 2).reshape(-1,1)

In [0]:
# Forecast
n_periods = 24 # this is 2 years
fitted, confint = sxmodel.predict(n_periods=n_periods, 
                                  #adding the exogenous variable index into the prediction 
                                  exogenous=np.tile(seasonal_index.seasonal, 2).reshape(-1,1), 
                                  return_conf_int=True)

index_of_fc = pd.date_range(df1.index[-1], periods = n_periods, freq='MS')  
index_of_fc                                

In [0]:
fitted

In [0]:
confint

In [0]:
# make series for plotting purpose
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

In [0]:
# Plot
plt.plot(df1['Passengers'])
plt.plot(fitted_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("SARIMAX Forecast of a10 - Airpassengers")
plt.show()